In [1]:
# combine all the file into one - Done

# write the function to take lookback window from 2 to 14 and calculate the corresponding pnl  - Done

# check each sector's pnl (later)

# add interaction as the threshold - Done

# remove all the extra strings  - Done

In [37]:
cd January

/Users/herculesgao/Desktop/Information_Tracer/January


In [38]:
import glob
import pandas as pd
import numpy as np
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import requests
import bs4 as bs

analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score



df_stock = pd.DataFrame()
for file_name in glob.glob('SPY500_stock*2024*.csv'):
    x = pd.read_csv(file_name, low_memory=False)
    df_stock = pd.concat([df_stock,x],axis=0)
try: 
    df_stock = df_stock.drop(columns = 'Unnamed: 0')
except:
    print("No unnamed column")
df_stock.sort_values(['Symbol', 'Date'], inplace=True)

df_temp = df_stock.groupby(['Symbol']).agg(numStock = ('Symbol', 'count')).reset_index()
days = df_stock['Date'].nunique()

excludedSymbol = list(df_temp[df_temp['numStock']< days]['Symbol'])
if excludedSymbol:
    print("we need to remove the following stocks that does not have the same days", excludedSymbol)
    df_stock = df_stock[~df_stock['Symbol'].isin(excludedSymbol)].reset_index(drop = True)
# remove null 

null_list = list(set(df_stock[df_stock['Posts_t'].isnull()]['Symbol']))
if null_list: 
    print("we need to remove the following stocks that does not have null", null_list)
    df_stock = df_stock[~df_stock['Symbol'].isin(null_list)].reset_index(drop = True)

we need to remove the following stocks that does not have the same days ['AEE', 'BG', 'CDAY', 'DAY', 'IEX', 'MHK', 'MLM', 'TDY', 'VLTO', 'VMC', 'WTW']


In [39]:
df_stock

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,ImpliedVolcalls,ImpliedVolputs,company,Sentiment_Vader,Posts_t,interaction,Changes
0,2024-01-02,A,138.789993,138.789993,140.589996,137.910004,138.190002,1165201.0,0.259651,0.247566,Agilent Technologies,0.0000,[],0.0,-0.240005
1,2024-01-03,A,131.160004,131.160004,137.440002,131.065002,138.000000,2074454.0,0.301399,0.251716,Agilent Technologies,0.0000,[],0.0,-7.589996
2,2024-01-04,A,131.000000,131.000000,131.494995,130.240005,130.550003,2446585.0,0.310310,0.277351,Agilent Technologies,0.0000,[],0.0,-7.750000
3,2024-01-05,A,130.559998,130.559998,131.960007,128.619995,130.000000,1327687.0,0.323981,0.287361,Agilent Technologies,0.0000,[],0.0,-8.190002
4,2024-01-08,A,133.380005,133.380005,133.570007,129.809998,130.139999,1247255.0,0.366461,0.314216,Agilent Technologies,0.0000,[],0.0,2.820007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9855,2024-01-24,ZTS,186.070007,186.070007,189.696396,185.929993,188.809998,1500294.0,0.318854,0.265022,Zoetis,0.8750,[{'d': 'Will Zoetis Inc Deliver Long-Term Retu...,5.0,-2.029999
9856,2024-01-26,ZTS,191.535004,191.535004,192.880005,188.970001,189.210007,1538561.0,0.000000,0.000000,Zoetis,0.0000,[],0.0,4.214996
9857,2024-01-29,ZTS,192.759995,192.759995,193.000000,189.960007,191.360001,1669900.0,0.000010,0.062509,Zoetis,0.7269,[{'d': 'Weekly Snacks #18\n\nEquities\n- Nvidi...,3.0,NaN
9858,2024-01-30,ZTS,191.429993,191.429993,193.509995,190.440002,192.869995,1607500.0,0.000010,0.062509,Zoetis,0.0000,[{'d': 'Pierwsze w tym roku zakupy w portfelu ...,13.0,NaN


In [40]:
df_stock.to_csv("SPY500_stock_2024_01_02_01_31.csv", index = False)

In [41]:
#text-preprocessing
def tweets_processing(df_stock):
    tweetsT = []
    sentiment_score = []
    
    for j in range(len(df_stock['Posts_t'])):
        tweets = df_stock.iloc[j]['Posts_t']
#         print(tweets)
        tweets_temp = []
        if len(tweets) >2:
            tweets = tweets.split("\'},")
            minS, maxS = 0,0
#             print(j)
            for i in range(len(tweets)):
 
                test_tw = re.split(" https://t",re.split("'d': |'i':",tweets[i])[1][1:-1])[0] # remove html and extra information
                test_tw = test_tw[:-1].replace("\\n", " ")  # remove newline 
                test_tw = test_tw.replace("amp;", " ") # remove ampersand
                new_score = sentiment_analyzer_scores(test_tw)['compound']
                if new_score< minS: 
                    minS = new_score
                elif new_score > maxS:
                    maxS = new_score
                tweets_temp.append(test_tw)
            res = maxS+ minS
            sentiment_score.append(res)
        else:
            sentiment_score.append(0.0)

        tweetsT.append(tweets_temp)
    df_stock['Tweets'] = tweetsT
    df_stock['Sentiment_Vader_corrected'] = sentiment_score
    return df_stock


df_stock = tweets_processing(df_stock)




In [42]:
df_stock

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,ImpliedVolcalls,ImpliedVolputs,company,Sentiment_Vader,Posts_t,interaction,Changes,Tweets,Sentiment_Vader_corrected
0,2024-01-02,A,138.789993,138.789993,140.589996,137.910004,138.190002,1165201.0,0.259651,0.247566,Agilent Technologies,0.0000,[],0.0,-0.240005,[],0.0000
1,2024-01-03,A,131.160004,131.160004,137.440002,131.065002,138.000000,2074454.0,0.301399,0.251716,Agilent Technologies,0.0000,[],0.0,-7.589996,[],0.0000
2,2024-01-04,A,131.000000,131.000000,131.494995,130.240005,130.550003,2446585.0,0.310310,0.277351,Agilent Technologies,0.0000,[],0.0,-7.750000,[],0.0000
3,2024-01-05,A,130.559998,130.559998,131.960007,128.619995,130.000000,1327687.0,0.323981,0.287361,Agilent Technologies,0.0000,[],0.0,-8.190002,[],0.0000
4,2024-01-08,A,133.380005,133.380005,133.570007,129.809998,130.139999,1247255.0,0.366461,0.314216,Agilent Technologies,0.0000,[],0.0,2.820007,[],0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9855,2024-01-24,ZTS,186.070007,186.070007,189.696396,185.929993,188.809998,1500294.0,0.318854,0.265022,Zoetis,0.8750,[{'d': 'Will Zoetis Inc Deliver Long-Term Retu...,5.0,-2.029999,[Will Zoetis Inc Deliver Long-Term Returns? $Z...,0.8750
9856,2024-01-26,ZTS,191.535004,191.535004,192.880005,188.970001,189.210007,1538561.0,0.000000,0.000000,Zoetis,0.0000,[],0.0,4.214996,[],0.0000
9857,2024-01-29,ZTS,192.759995,192.759995,193.000000,189.960007,191.360001,1669900.0,0.000010,0.062509,Zoetis,0.7269,[{'d': 'Weekly Snacks #18\n\nEquities\n- Nvidi...,3.0,NaN,[Weekly Snacks #18 Equities - Nvidia $NVDA - ...,0.7269
9858,2024-01-30,ZTS,191.429993,191.429993,193.509995,190.440002,192.869995,1607500.0,0.000010,0.062509,Zoetis,0.0000,[{'d': 'Pierwsze w tym roku zakupy w portfelu ...,13.0,NaN,[Pierwsze w tym roku zakupy w portfelu dolarow...,0.0000


In [43]:

def look_back_sentiment(df_stock, org = True, lookback_window=1):
    n = df_stock['Date'].nunique()
    Sentiment_Vader_2 = []
#     t = 0 
    for i in range(0,len(df_stock['Posts_t']),n):
        for j in range(n):
            if (j - lookback_window ) >=0:
                past_days = df_stock.iloc[i+j-lookback_window:i+j]
                if org:
                    sentiment_now =  past_days[past_days['Posts_t'].str.len()>2]['Sentiment_Vader'].mean()
                else:
                    sentiment_now =  past_days[past_days['Posts_t'].str.len()>2]['Sentiment_Vader_corrected'].mean()
                if np.isnan(sentiment_now):  #find all the empty twitter list
                    Sentiment_Vader_2.append(0.0)
                else:         
                    Sentiment_Vader_2.append(sentiment_now)
            else:  # outside the lookback window days 
                #base case 
                Sentiment_Vader_2.append(0.0)
    df_stock['Sentiment_Vader_lookback'] = Sentiment_Vader_2
    return df_stock


In [44]:

def VADER_Sentiment_report(df_stock, lookback_window = 1, vader_threshold = 0.05,
                           interact_threshold = 0, org = True):
    pnl = 0 
    stock_pnl = []
    stock_names = []
    len_tweets = []
    interactions = []
    sumA = 0 
    sumTP = 0 
    sumTN = 0
    sumFP = 0 
    sumFN = 0 
    sumWrong = 0 
    sumPlus = 0 
    df_stock = look_back_sentiment(df_stock, org, lookback_window)
    n = df_stock['Date'].nunique()
    for i in range(0,len(df_stock['Symbol']),n): 
        symbol = df_stock['Symbol'][i]
        temp_hold = 0
        long = False
        short = False
        total_pnl = 0 
        total_t = 0
        num = 0 
        num_s = 0 
        interaction = 0 
        for j in range(n):
            change = df_stock['Changes'][i+j]
            interaction += df_stock['interaction'][i+j]
            tw = df_stock['Tweets'][i+j]
#             print(tw, len(tw))
            num_tw = len(tw)
#             print(i+j, df_stock['Sentiment_Vader_lookback'][i+j])
            if df_stock['Sentiment_Vader_lookback'][i+j] >= vader_threshold and df_stock['interaction'][i+j] >= interact_threshold:
                # no position
                if not long and not short:  # the first buy
                    temp_hold = df_stock['Adj Close'][i+j]
                    num+=1
                    long = True
                # have a short position and close the short position
                elif short: 
                    total_pnl += (temp_hold + df_stock['Adj Close'][i+j])
                    short = False
                    num_s = 0 
                elif long and not short: 
                    num+=1
                    temp_hold += df_stock['Adj Close'][i+j]

            elif df_stock['Sentiment_Vader_lookback'][i+j] <= - vader_threshold and df_stock['interaction'][i+j] >= interact_threshold:
                # have a long position and close the long position 
                if long and not short: 
#                         print("sell signal and exit")
                    total_pnl += (df_stock['Adj Close'][i+j] -  (temp_hold/num))*1
                    long = False
                    num = 0 
                # do not have long or short position
                elif not long and not short: 
                    temp_hold = - df_stock['Adj Close'][i+j]
                    short = True
                    num_s = 1
#                 # do not have a long position and already short a stock
                elif not long and short: 
#                         print("short again")
                    temp_hold = (- df_stock['Adj Close'][i+j]+ temp_hold)/(num_s+1)
                    num_s +=1

            if long and not short and j == n-1: 
                total_pnl += (df_stock['Adj Close'][i+j] - (temp_hold/num))*1
            elif short and not long and j == n-1: 
                total_pnl += ((df_stock['Adj Close'][i+j] + temp_hold)/(num_s+1))*1
            if change > 0: sumPlus += 1
            if (df_stock['Sentiment_Vader_lookback'][i+j] >= vader_threshold and change > 0) or (df_stock['Sentiment_Vader_lookback'][i+j] <= - vader_threshold  and float(change) < 0): 
                sumA += 1
                if df_stock['Sentiment_Vader_lookback'][i+j] >= vader_threshold and change > 0:
                    sumTP +=1
                else:
                    sumTN +=1
            if (df_stock['Sentiment_Vader_lookback'][i+j] >=  vader_threshold  and change < 0) or (df_stock['Sentiment_Vader_lookback'][i+j] <= - vader_threshold  and float(change) > 0): 
                sumWrong += 1
                if df_stock['Sentiment_Vader_lookback'][i+j] >= vader_threshold and change < 0:
                    sumFP +=1
                else:
                    sumFN +=1
        stock_names.append(df_stock['Symbol'][i])
        len_tweets.append(num_tw)
        interactions.append(interaction/n)
        pnl += total_pnl
        stock_pnl.append(total_pnl)
    stock = sorted(set(df_stock['Symbol']))
    df_pnl = pd.DataFrame({'Symbol': stock_names, 'PNL_VADER':stock_pnl, 'Len_Ts':len_tweets,'Interactions':interactions} )
    l_hProfits = []
    for i in range(0,len(df_stock['Symbol']),n): 
        l_hProfit = (df_stock['Adj Close'][i+n-1] - df_stock['Adj Close'][i])
        l_hProfits.append(l_hProfit)
    df_pnl['PNL_LongnHold'] = l_hProfits
    
    # add sectors 

    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    table.findAll('tr')[1:][0].findAll('td')[1].text
    tickers = []
    sectors = []
    dc = {}

    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        sector = row.findAll('td')[2].text
        dc[ticker.replace('\n', '').replace('.', '-')] = sector

    extra_list = []

    sectors = []
    for i in df_pnl['Symbol']:
        try:
    #         print(i,dc[i])
            if i == 'CDAY':
              sectors.append('Information Technology') 
            else:
                sectors.append(dc[i])
        except: 
            
            sec = """
            Sector Reference table: 

             'Health Care',
             'Industrials',
             'Information Technology',
             'Consumer Discretionary',
             'Financials',
             'Consumer Staples',
             'Utilities',
             'Materials',
             'Real Estate',
             'Energy',
             'Communication Services'

             """
            print("we could not find the sector for the following!",i)
            print(sec)
            add_sector = input("Please give a sector for the ticker below!")
            
            extra_list.append((i,add_sector))
            sectors.append(add_sector)
    df_pnl['Sector'] = sectors
    total_vader_num = len(df_stock[df_stock['Sentiment_Vader_corrected']!=0])
    accuracy = sumA/total_vader_num
    f1 = sumTP/(sumTP + 0.5*(sumFP + sumTN))

    return pnl, df_pnl,accuracy,f1


In [46]:
lookback_days = []
accuracys = []
f1s = []
pnls = []
orgins = []
interactions_thres = []
vader_thres = []
total_pnldfs = []
month = 'Jan'
print("There are ",df_stock['Date'].nunique(),"unique days in",month)

for i in range(2,15):
    print("Different days (between 2-14)", i)
    lookback_days.append(i)
    pnl, df_pnl,accuracy,f1 = VADER_Sentiment_report(df_stock, i,
                                                     vader_threshold = 0.01,
                                                     interact_threshold = 0,
                                                     org = True)
    print("Orginal", pnl)

    accuracys.append(accuracy)
    f1s.append(f1)
    pnls.append(pnl)
    interactions_thres.append(0)
    vader_thres.append(0.01)
    orgins.append(True)
    total_pnldfs.append(df_pnl)
    pnl, df_pnl,accuracy,f1 = VADER_Sentiment_report(df_stock, i,
                                                     vader_threshold = 0.01,
                                                     interact_threshold = 0,
                                                     org = False)
    print("Text-preprocessed", pnl)
    lookback_days.append(i)
    accuracys.append(accuracy)
    f1s.append(f1)
    pnls.append(pnl)
    interactions_thres.append(0)
    vader_thres.append(0.01)
    orgins.append(False)
    total_pnldfs.append(df_pnl)
    
df_sum = pd.DataFrame({'lookback_days':lookback_days,
                        'accuracy':accuracys,'f1':f1s,
                        'pnl':pnls,
                        'orgin':orgins,
                        'interactions_thres':interactions_thres,
                        'vader_thres':vader_thres})

There are  20 unique days in Jan
Different days (between 2-14) 2
Orginal 5370.904605221649
Text-preprocessed 5398.923127371423
Different days (between 2-14) 3
Orginal 7705.684755247657
Text-preprocessed 7566.822080037585
Different days (between 2-14) 4
Orginal 8222.539014613756
Text-preprocessed 7994.758482094377
Different days (between 2-14) 5
Orginal 7160.27946700009
Text-preprocessed 7076.962328323892
Different days (between 2-14) 6
Orginal 6525.394089739993
Text-preprocessed 6479.660657254084
Different days (between 2-14) 7
Orginal 5316.194463964747
Text-preprocessed 5347.078336059988
Different days (between 2-14) 8
Orginal 4876.436291823433
Text-preprocessed 5031.684818140069
Different days (between 2-14) 9
Orginal 4494.301283361344
Text-preprocessed 4553.455406889593
Different days (between 2-14) 10
Orginal 3928.7202615007586
Text-preprocessed 4113.522693118602
Different days (between 2-14) 11
Orginal 2593.3269394940935
Text-preprocessed 2590.916415439535
Different days (between 

In [61]:
df_sum

,lookback_days,accuracy,f1,pnl,orgin,interactions_thres,vader_thres
0,2,0.413144,0.610896,5370.904605,True,0,0.01
1,2,0.409493,0.609648,5398.923127,False,0,0.01
2,3,0.426595,0.627837,7705.684755,True,0,0.01
3,3,0.421983,0.626364,7566.822080,False,0,0.01
4,4,0.422560,0.640417,8222.539015,True,0,0.01
5,4,0.419101,0.640257,7994.758482,False,0,0.01
6,5,0.396618,0.618375,7160.279467,True,0,0.01
7,5,0.393159,0.618227,7076.962328,False,0,0.01
8,6,0.376249,0.641055,6525.394090,True,0,0.01
9,6,0.375096,0.640136,6479.660657,False,0,0.01


In [62]:
print("Long and Hold pnl", sum(df_pnl['PNL_LongnHold']))

Long and Hold pnl 1266.6167697906494


In [63]:
# 3 the most profitable one
pnl, df_pnl,accuracy,f1 = VADER_Sentiment_report(df_stock, 4, 0.01, 0, True)

In [64]:
df_pnl.describe()

,PNL_VADER,Len_Ts,Interactions,PNL_LongnHold
count,493.000000,493.000000,493.000000,493.000000
mean,16.678578,21.170385,491.089351,2.569202
std,56.045059,68.137888,5633.587152,21.846207
min,-48.886253,0.000000,0.000000,-92.399994
25%,-0.874377,1.000000,0.750000,-4.180000
50%,0.913131,2.000000,5.850000,-0.659996
75%,9.040795,9.000000,39.400000,5.005005
max,704.134089,500.000000,115889.050000,192.429932


In [65]:
sum(df_pnl['PNL_LongnHold']),sum(df_pnl['PNL_VADER'])

(1266.6167697906494, 8222.539014613756)

In [66]:
def get_df_summary(df,df_stock):
    l_hProfits = []
    pctVaderPNL = []
    pctLHPNL = []
    total_cost = 0

    nuni = df_stock['Date'].nunique()
    for i in range(0,len(df_stock['Symbol']),nuni): 
    #     print(df_stock['Symbol'][i], df_stock['Symbol'][i+5])
        l_hProfit = df_stock['Adj Close'][i+nuni-1] - df_stock['Adj Close'][i]
        l_hProfits.append(l_hProfit)
      

    df['PNL_LongnHold'] = l_hProfits
    
    for j in range(0,len(df_stock['Symbol']),nuni): 
        stock_vader = df[df['Symbol']==df_stock['Symbol'][j]]['PNL_VADER']
        stock_lh = df[df['Symbol']==df_stock['Symbol'][j]]['PNL_LongnHold']
        pnl = stock_vader.loc[stock_vader.index[0]]
        pnl_lh = stock_lh.loc[stock_lh.index[0]]
        pctVaderPNL.append((pnl/df_stock['Adj Close'][j])*100)
        pctLHPNL.append((pnl_lh/df_stock['Adj Close'][j])*100)
        total_cost += df_stock['Adj Close'][i]
    print("PNL of VADER", sum(df['PNL_VADER']))
    
    print("PNL of L/H", sum(df['PNL_LongnHold']))

    df['pctVaderPNL'] = pctVaderPNL
    df['pctLHPNL'] = pctLHPNL
    
    print("PNL Percentage of VADER", np.mean(df['pctVaderPNL']))
    print("PNL Percentage of L/H", np.mean(df['pctLHPNL']))
    
    df_summary = df.groupby('Sector', group_keys=True, as_index = False).agg(
        avgPctVADER=('pctVaderPNL', np.mean),
        avgPctL_H=('pctLHPNL', np.mean),
        sumVADER=('PNL_VADER', np.sum),
        sumL_H=('PNL_LongnHold', np.sum),
        sumTwts =('Len_Ts', np.sum),
        CountStocks=('Symbol', 'nunique'))
    return df, df_summary,total_cost, (np.mean(df['pctVaderPNL'])>np.mean(df['pctLHPNL']))

In [67]:
df, df_summary,total_cost,res = get_df_summary(df_pnl,df_stock)

PNL of VADER 8222.539014613756
PNL of L/H 1266.6167697906494
PNL Percentage of VADER 9.12505786187774
PNL Percentage of L/H -0.7703966373278234


In [68]:
total_cost

96923.80300903319

In [69]:
excess_l_h_return = sum(df_pnl['PNL_LongnHold'])/total_cost 
excess_l_h_return

0.01306817036133634

In [70]:
excess_vader_return = sum(df_pnl['PNL_VADER'])/total_cost
excess_vader_return

0.08483508446163039

In [57]:
((0.6413453388548358/0.1552738109673628) -1)*100

313.0415392391193

In [58]:
#SPY index growth

((4954.23/4288.36)-1)*100

15.527381096736281

In [59]:
risk_free_return = 0.0522

In [74]:
sigma_a = 0.017927704424869358

In [73]:
np.std([0.09273696647187206,  0.1302448373498233,  0.09057778660135926, 0.08483508446163039 ])

0.017927704424869358

In [75]:
sharpe_r = (excess_vader_return - risk_free_return)/sigma_a
sharpe_r

1.8203716264062741

In [76]:
df[df['Sector']=='Communication Services']

,Symbol,PNL_VADER,Len_Ts,Interactions,PNL_LongnHold,Sector,pctVaderPNL,pctLHPNL
91,CHTR,262.977322,2,3.40,-20.800018,Communication Services,67.170012,-5.312768
97,CMCSA,1.463000,9,76.30,2.870003,Communication Services,3.350125,6.572024
134,DIS,71.794944,18,68.25,5.330002,Communication Services,79.139046,5.875222
146,EA,47.224998,32,25.90,1.800003,Communication Services,34.780527,1.325676
188,FOX,0.000000,0,0.00,1.870001,Communication Services,0.000000,6.645348
189,FOXA,1.357142,0,0.00,2.059999,Communication Services,4.487904,6.812168
204,GOOG,16.153584,0,0.25,2.240005,Communication Services,11.574652,1.605048
205,GOOGL,87.446660,0,1.25,1.930008,Communication Services,63.289181,1.396836
241,IPG,0.000000,2,0.00,-0.109997,Communication Services,0.000000,-0.332317
287,LYV,-2.773754,0,0.90,-2.950005,Communication Services,-3.021519,-3.213513


In [77]:
df_summary

,Sector,avgPctVADER,avgPctL_H,sumVADER,sumL_H,sumTwts,CountStocks
0,Communication Services,17.204490,2.998542,619.589899,146.414980,660,22
1,Consumer Discretionary,11.426304,-1.631056,644.412874,447.980263,2724,52
2,Consumer Staples,10.133011,-2.303066,289.028389,-16.130013,450,37
3,Energy,0.111778,-3.334867,29.900948,-24.303347,166,23
4,Financials,10.325220,0.946901,974.407381,203.138280,1676,71
5,Health Care,9.761156,-0.180961,2348.873343,10.564429,1462,64
6,Industrials,9.169544,0.217697,1595.675941,124.559588,832,75
7,Information Technology,9.438488,2.921076,1195.126956,773.233974,2061,63
8,Materials,11.328689,-5.368882,421.943564,-117.830051,214,26
9,Real Estate,1.123616,-5.725092,0.803551,-198.269943,95,31


In [78]:
# pctage of pnl 
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# for i in range(len(summary)):
week =3

labels = list(df_summary["Sector"])

colors = ['green','red','wheat','powderblue','salmon','brown','pink','darkblue','sienna','lightgreen','lightskyblue']
fig = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5],
                    subplot_titles=("VADER PnL by Sectors with Sentiment Threshold = ±0.01","L/H PnL by Sectors"),
                    vertical_spacing=0.1,
                  horizontal_spacing=0.1,
                  shared_yaxes='all')
fig.add_trace(go.Bar(x = df_summary["Sector"], y=df_summary["avgPctVADER"], marker_color=colors),
              row=1, col=1)

fig.add_trace(go.Bar(x = df_summary["Sector"], y=df_summary["avgPctL_H"],marker_color=colors),
              row=1, col=2)
fig.update_traces(texttemplate='%{y:.2f}')
fig.update_layout(title="Lookback Days-"+str(week)+': PNL percentage comparison',
                   xaxis_title='Sector Name',
                   yaxis_title='PnL ($)')
fig.show()

In [79]:
# pnl 



labels = list(df_summary["Sector"])
colors = ['green','red','wheat','powderblue','salmon','brown','pink','darkblue','sienna','lightgreen','lightskyblue']
fig = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5],
                    subplot_titles=("VADER PnL by Sectors with Sentiment Threshold = ±0.01","L/H PnL by Sectors"),
                    vertical_spacing=0.1,
                  horizontal_spacing=0.1,
                  shared_yaxes='all')
fig.add_trace(go.Bar(x = df_summary["Sector"], y=df_summary["sumVADER"], marker_color=colors),
              row=1, col=1)

fig.add_trace(go.Bar(x = df_summary["Sector"], y=df_summary["sumL_H"],marker_color=colors),
              row=1, col=2)
fig.update_traces(texttemplate='%{y:.2f}')
fig.update_layout(title="Lookback Days-"+str(week)+': PNL comparison',
                   xaxis_title='Sector Name',
                   yaxis_title='PnL ($)')
fig.show()

In [80]:
fig = px.imshow(df_summary[['avgPctVADER','avgPctL_H','sumVADER','sumL_H','sumTwts','CountStocks']].corr(), text_auto=True, aspect="auto")
fig.update_layout(title="Lookback days-"+str(week)+': All tickers PNL each factor in-between correlation', xaxis_nticks=36)
fig.show()

In [119]:
text = "$A Agilent Technologies current stock price $130\\n\\nLow of $130.01 \\nP/E  (price-to-earnings ratio) 31.09 \\nAnnuel EPS grow past 5 years 33%\\nEPS Q/Q (quarterly earnings growth) 31%\\n\\nClosed yesterday at $133 - needs to pull back to $113 to close that gap. LT Puts? https://t.co/QJZBNU7kbW'  "



sentiment_analyzer_scores(text)

{'neg': 0.0, 'neu': 0.94, 'pos': 0.06, 'compound': 0.3818}

In [120]:
text1 = "$A Agilent Technologies current stock price $130 Low of $130.01 P/E (price-to-earnings ratio) 31.09 Annuel EPS grow past 5 years 33%\\nEPS Q/Q (quarterly earnings growth) 31% Closed yesterday at $133 - needs to pull back to $113 to close that gap. LT Puts?"

sentiment_analyzer_scores(text1)

{'neg': 0.046, 'neu': 0.897, 'pos': 0.057, 'compound': 0.128}

In [121]:
text1 = "A Agilent Technologies current stock price $130 Low of $130.01 P/E (price-to-earnings ratio) 31.09 Annuel EPS grow past 5 years 33%\\nEPS Q/Q (quarterly earnings growth) 31% Closed yesterday at $133 - needs to pull back to $113 to close that gap. LT Puts?"

sentiment_analyzer_scores(text1)

{'neg': 0.046, 'neu': 0.897, 'pos': 0.057, 'compound': 0.128}